In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import gc
import datetime as dt
import seaborn as sns
import matplotlib.gridspec as gridspec
import ipaddress
import random as rnd
import plotly.graph_objs as go
import lime
import lime.lime_tabular
import itertools
import warnings
warnings.filterwarnings(action='once')
from pandas.tools.plotting import scatter_matrix
from functools import reduce
from numpy import genfromtxt
from scipy import linalg
from scipy.stats import multivariate_normal
from sklearn import preprocessing, mixture
from sklearn.metrics import classification_report, average_precision_score, f1_score, recall_score, precision_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

/usr/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning:

numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88



In [2]:
def data_cleasing(df):
    # data cleasing, feature engineering and save clean data into pickles
    
    print('### Data Cleasing and Feature Engineering')
    df.head()
    le = preprocessing.LabelEncoder()
    
    # [Protocol] - Discard ipv6-icmp and categorize
    # df = df[df.Proto != 'ipv6-icmp']
    df['Proto'] = df['Proto'].fillna('-')
    df['Proto'] = le.fit_transform(df['Proto'])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

    # [Label] - Categorize 
    anomalies = df.Label.str.contains('Botnet')
    normal = np.invert(anomalies);
    df.loc[anomalies, 'Label'] = np.uint8(1)
    df.loc[normal, 'Label'] = np.uint8(0)
    df['Label'] = pd.to_numeric(df['Label'])

    # [Dport] - replace NaN with 0 port number
    df['Dport'] = df['Dport'].fillna('0')
    df['Dport'] = df['Dport'].apply(lambda x: int(x,0))

    # [sport] - replace NaN with 0 port number
    try:
        df['Sport'] = df['Sport'].fillna('0')
        df['Sport'] = df['Sport'].str.replace('.*x+.*', '0')
        df['Sport'] = df['Sport'].apply(lambda x: int(x,0))
    except:                        
        print("### data_cleasing: Unexpected error:", sys.exc_info()[0])

    # [sTos] - replace NaN with "10" and convert to int
    df['sTos'] = df['sTos'].fillna('10')
    df['sTos'] = df['sTos'].astype(int)

    # [dTos] - replace NaN with "10" and convert to int
    df['dTos'] = df['dTos'].fillna('10')
    df['dTos'] = df['dTos'].astype(int)

    # [State] - replace NaN with "-" and categorize
    df['State'] = df['State'].fillna('-')
    df['State'] = le.fit_transform(df['State'])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

    # [Dir] - replace NaN with "-" and categorize 
    df['Dir'] = df['Dir'].fillna('-')
    df['Dir'] = le.fit_transform(df['Dir'])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

    # [SrcAddr] Extract subnet features and categorize
    df['SrcAddr'] = df['SrcAddr'].fillna('0.0.0.0')
    # tmp_df = pd.DataFrame(df['SrcAddr'].str.split('.').tolist(), columns = ['1','2','3','4'])
    # df["SrcAddr1"] = tmp_df["1"]
    # df["SrcAddr2"] = tmp_df["1"].map(str) + tmp_df["2"]
    # df["SrcAddr3"] = tmp_df["1"].map(str) + tmp_df["2"].map(str) + tmp_df["3"]
    # df['SrcAddr0'] = le.fit_transform(df['SrcAddr'])
    # le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    # df['SrcAddr1'] = df['SrcAddr1'].astype(int)
    # df['SrcAddr2'] = df['SrcAddr2'].astype(int)
    # df['SrcAddr3'] = df['SrcAddr3'].astype(int)

    # [DstAddr] Extract subnet features
    df['DstAddr'] = df['DstAddr'].fillna('0.0.0.0')
    # tmp_df = pd.DataFrame(df['DstAddr'].str.split('.').tolist(), columns = ['1','2','3','4'])
    # df["DstAddr1"] = tmp_df["1"]
    # df["DstAddr2"] = tmp_df["1"].map(str) + tmp_df["2"]
    # df["DstAddr3"] = tmp_df["1"].map(str) + tmp_df["2"].map(str) + tmp_df["3"]
    # df['DstAddr0'] = le.fit_transform(df['DstAddr'])
    # df['DstAddr1'] = df['DstAddr1'].astype(int)
    # df['DstAddr2'] = df['DstAddr2'].astype(int)
    # df['DstAddr3'] = df['DstAddr3'].astype(int)

    # [StartTime] - Parse to datatime, reindex based on StartTime, but first drop the ns off the time stamps
    df['StartTime'] = df['StartTime'].apply(lambda x: x[:19])
    df['StartTime'] = pd.to_datetime(df['StartTime'])
    df = df.set_index('StartTime')
    
    return df

def classify_ip(ip):
    '''
    str ip - ip address string to attempt to classify. treat ipv6 addresses as N/A
    '''
    try: 
        ip_addr = ipaddress.ip_address(ip)
        if isinstance(ip_addr, ipaddress.IPv6Address):
            return 'ipv6'
        elif isinstance(ip_addr, ipaddress.IPv4Address):
            # split on .
            octs = ip_addr.exploded.split('.')
            if 0 < int(octs[0]) < 127: return 'A'
            elif 127 < int(octs[0]) < 192: return 'B'
            elif 191 < int(octs[0]) < 224: return 'C'
            else: return 'N/A'
    except ValueError:
        return 'N/A'
            
def avg_duration(x):
    return np.average(x)
            
def n_dports_gt1024(x):
    if x.size == 0: return 0
    return reduce((lambda a,b: a+b if b>1024 else a),x)
n_dports_gt1024.__name__ = 'n_dports>1024'

def n_dports_lt1024(x):
    if x.size == 0: return 0
    return reduce((lambda a,b: a+b if b<1024 else a),x)
n_dports_lt1024.__name__ = 'n_dports<1024'

def n_sports_gt1024(x):
    if x.size == 0: return 0
    return reduce((lambda a,b: a+b if b>1024 else a),x)
n_sports_gt1024.__name__ = 'n_sports>1024'

def n_sports_lt1024(x):
    if x.size == 0: return 0
    return reduce((lambda a,b: a+b if b<1024 else a),x)
n_sports_lt1024.__name__ = 'n_sports<1024'

def label_atk_v_norm(x):
    for l in x:
        if l == 1: return 1
    return 0
label_atk_v_norm.__name__ = 'label'

def background_flow_count(x):
    count = 0
    for l in x:
        if l == 0: count += 1
    return count

def normal_flow_count(x):
    if x.size == 0: return 0
    count = 0
    for l in x:
        if l == 0: count += 1
    return count

def n_conn(x):
    return x.size

def n_tcp(x):
    count = 0
    for p in x: 
        if p == 10: count += 1 # tcp == 10
    return count
            
def n_udp(x):
    count = 0
    for p in x: 
        if p == 11: count += 1 # udp == 11
    return count
            
def n_icmp(x):
    count = 0
    for p in x: 
        if p == 1: count += 1 # icmp == 1
    return count

def n_s_a_p_address(x):
    count = 0
    for i in x: 
        if classify_ip(i) == 'A': count += 1
    return count
            
def n_d_a_p_address(x):
    count = 0
    for i in x: 
        if classify_ip(i) == 'A': count += 1
    return count

def n_s_b_p_address(x):
    count = 0
    for i in x: 
        if classify_ip(i) == 'B': count += 1
    return count

def n_d_b_p_address(x):
    count = 0
    for i in x: 
        if classify_ip(i) == 'A': count += 1
    return count
                        
def n_s_c_p_address(x):
    count = 0
    for i in x: 
        if classify_ip(i) == 'C': count += 1
    return count
            
def n_d_c_p_address(x):
    count = 0
    for i in x: 
        if classify_ip(i) == 'C': count += 1
    return count
                        
def n_s_na_p_address(x):
    count = 0
    for i in x: 
        if classify_ip(i) == 'N/A': count += 1
    return count
            
def n_d_na_p_address(x):
    count = 0
    for i in x: 
        if classify_ip(i) == 'N/A': count += 1
    return count

def n_ipv6(x):
    count = 0
    for i in x:
        if classify_ip(i) == 'ipv6': count += 1
    return count

def estimateGaussian(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.cov(dataset.T)
    return mu, sigma

def multivariateGaussian(dataset, mu, sigma):
    p = multivariate_normal(mean=mu, cov=sigma, allow_singular=True)
    return p.pdf(dataset)

def selectThresholdByCV(probs, labels):
    # select best epsilon (threshold)
    
    # initialize
    best_epsilon = 0
    best_f1 = 0
    best_precision = 0
    best_recall = 0
    
    min_prob = min(probs);
    max_prob = max(probs);
    stepsize = (max(probs) - min(probs)) / 1000;
    epsilons = np.arange(min(probs), max(probs), stepsize)
    
    for epsilon in epsilons:
        predictions = (probs < epsilon)
        
        f1 = f1_score(labels, predictions, average = "binary")
        Recall = recall_score(labels, predictions, average = "binary")
        Precision = precision_score(labels, predictions, average = "binary")            
        
        if f1 > best_f1:
            best_epsilon = epsilon
            best_f1 = f1
            best_precision = Precision
            best_recall = Recall

    return best_f1, best_epsilon


def print_classification_report(y_test, y_predic):
#     print('### Classification report:')
#     print(classification_report(y_test, y_predic))
    
#     print('\tAverage Precision = ' + str(average_precision_score(y_test, y_predic)))
    
    f = f1_score(y_test, y_predic, average = "binary")
    Recall = recall_score(y_test, y_predic, average = "binary")
    Precision = precision_score(y_test, y_predic, average = "binary")
    print('### Binary F1 Score, Recall and Precision:')
    print('\tF1 Score %f' %f)
    print('\tPrecision Score %f' %Precision)
    print('\tRecall Score %f' %Recall)
    

def model_order_selection(data, max_components):
    bic = []
    lowest_bic = np.infty
    n_components_range = range(1, max_components)
    cov_types = ['spherical', 'tied', 'diag', 'full']
    
    for cov_type in cov_types:
        for n_components in n_components_range:
            gmm = mixture.GaussianMixture(n_components=n_components, covariance_type=cov_type)
            gmm.fit(data)
            bic.append(gmm.bic(data))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmm = gmm
                best_n_components = n_components
                best_cov_type = cov_type

    return best_n_components, best_cov_type

def data_splitting(df, drop_features):
    # Data splitting
    
    # drop non discriminant features
    df.drop(drop_features, axis =1, inplace = True)

    # split into normal and anomaly
    df_l1 = df[df["Label"] == 1]
    df_l0 = df[df["Label"] == 0]

    # Length and indexes
    anom_len = len(df_l1)                               # total number of anomalous flows
    anom_train_end = anom_len // 2                      # 50% of anomalous for training
    anom_cv_start = anom_train_end + 1                  # 50% of anomalous for testing
    norm_len = len(df_l0)                               # total number of normal flows
    norm_train_end = (norm_len * 60) // 100             # 60% of normal for training
    norm_cv_start = norm_train_end + 1                  # 20% of normal for cross validation
    norm_cv_end = (norm_len * 80) // 100                # 20% of normal for cross validation
    norm_test_start = norm_cv_end + 1                   # 20% of normal for testing

    # anomalies split data
    anom_cv_df  = df_l1[:anom_train_end]                # 50% of anomalies59452 
    anom_test_df = df_l1[anom_cv_start:anom_len]        # 50% of anomalies

    # normal split data
    norm_train_df = df_l0[:norm_train_end]              # 60% of normal
    norm_cv_df = df_l0[norm_cv_start:norm_cv_end]       # 20% of normal
    norm_test_df = df_l0 [norm_test_start:norm_len]     # 20% of normal

    # CV and test data. train data is norm_train_df
    cv_df = pd.concat([norm_cv_df, anom_cv_df], axis=0)
    test_df = pd.concat([norm_test_df, anom_test_df], axis=0)

    # labels
    cv_label = cv_df["Label"]
    test_label = test_df["Label"]

    # drop label
    norm_train_df = norm_train_df.drop(labels = ["Label"], axis = 1)
    cv_df = cv_df.drop(labels = ["Label"], axis = 1)
    test_df = test_df.drop(labels = ["Label"], axis = 1)
    
    return norm_train_df, cv_df, test_df, cv_label, test_label

In [ ]:
%%time
# features
column_types = {
            'StartTime': 'str',
            'Dur': 'float32',
            'Proto': 'str',
            'SrcAddr': 'str',
            'Sport': 'str',
            'Dir': 'str',
            'DstAddr': 'str',
            'Dport': 'str',
            'State': 'str',
            'sTos': 'float16',
            'dTos': 'float16',
            'TotPkts': 'uint32',
            'TotBytes': 'uint32',
            'SrcBytes': 'uint32',
            'Label': 'str'}

# feature selection
drop_features = {
    'drop_features01':['SrcAddr','DstAddr','sTos','Sport','Proto','TotBytes','SrcBytes'],
    'drop_features02':['SrcAddr','DstAddr','sTos','Sport','TotBytes','SrcBytes'],
    'drop_features03':['SrcAddr','DstAddr','sTos','Sport','Proto','SrcBytes'],
    'drop_features04':['SrcAddr','DstAddr','sTos','Proto']
}

# raw files
raw_path = os.path.join('/media/thiago/ubuntu/datasets/network/','stratosphere-botnet-2011/ctu-13/raw/')
raw_directory = os.fsencode(raw_path)
raw_files = os.listdir(raw_directory)
print("### Directory: ", raw_directory)
print("### Files: ", raw_files)

# pickle files - have the same names but different directory
pkl_path = os.path.join('/media/thiago/ubuntu/datasets/network/','stratosphere-botnet-2011/ctu-13/pkl/')
pkl_directory = os.fsencode(pkl_path)

# for each feature set
for features_key, value in drop_features.items():

    # initialize labels
    mgm_cv_label = []
    mgm_pred_cv_label = []
    mgm_test_label = []
    mgm_pred_test_label = []
    gmm_cv_label = []
    gmm_pred_cv_label = []
    gmm_test_label = []
    gmm_pred_test_label = []

    # for each file
    for sample_file in raw_files:
        pkl_file_path = os.path.join(pkl_directory, sample_file).decode('utf-8')
        raw_file_path = os.path.join(raw_directory, sample_file).decode('utf-8')

        # read pickle or raw dataset file with pandas
        if os.path.isfile(pkl_file_path):
            print("### Sample File: ", pkl_file_path)
            df = pd.read_pickle(pkl_file_path)
        else:
            print("### Sample File: ", raw_file_path)        
            raw_df = pd.read_csv(raw_file_path, header = 0, dtype=column_types)
            df = data_cleasing(raw_df)
        gc.collect()

        # data splitting
        norm_train_df, cv_df, test_df, cv_label, test_label = data_splitting(df, drop_features[features_key])

        # Scaler: raw, standardization (zero mean and unitary variance) or robust scaler
        scaler = 'RobustScaler'
        norm_train_values = RobustScaler().fit_transform(norm_train_df.values)
        norm_train_df = pd.DataFrame(norm_train_values, index=norm_train_df.index, columns=norm_train_df.columns)
        gc.collect()
        cv_values = RobustScaler().fit_transform(cv_df.values)
        cv_df = pd.DataFrame(cv_values, index=cv_df.index, columns=cv_df.columns)
        gc.collect()
        test_values = RobustScaler().fit_transform(test_df.values)
        test_df = pd.DataFrame(test_values, index=test_df.index, columns=test_df.columns)
        gc.collect()        

        try:
            # [MGM] estimate the mean vector and the covariance matrix from the normal data
            mu, sigma = estimateGaussian(norm_train_df)

            # [MGM] estimate CV
            p_cv = multivariateGaussian(cv_df, mu, sigma)

            # [MGM] estimate test
            p_test = multivariateGaussian(test_df, mu, sigma)

            # [MGM] threshold selection
            fscore, epsilon = selectThresholdByCV(p_cv, cv_label)

            # [MGM] Cross Validation
            pred_cv_label = (p_cv < epsilon)
            mgm_cv_label.extend(cv_label.astype(int))
            mgm_pred_cv_label.extend(pred_cv_label.astype(int))
            print('###',features_key,scaler,'[MGM]','Classification report for Cross Validation dataset')
            print_classification_report(cv_label, pred_cv_label)

            # [MGM] Test  
            pred_test_label = (p_test < epsilon)    
            mgm_test_label.extend(test_label.astype(int))
            mgm_pred_test_label.extend(pred_test_label.astype(int))
            print('###',features_key,scaler,'[MGM]','Classification report for Test dataset')
            print_classification_report(test_label, pred_test_label)
        except:                        
            print("### [MGM] Error!!")

        try:
            # [GMM] Fit a Gaussian Mixture Model
            # best_n_components, best_cov_type = model_order_selection(norm_train_df, len(norm_train_df.columns))
            gmm = mixture.GaussianMixture(n_components=10, covariance_type='full')
            gmm.fit(norm_train_df)

            # [GMM] Cross Validation and threshold selection
            p_cv = gmm.score_samples(cv_df)

            # [GMM] Cross Validation
            fscore, epsilon = selectThresholdByCV(p_cv, cv_label)
            pred_cv_label = (p_cv < epsilon)
            gmm_cv_label.extend(cv_label.astype(int))
            gmm_pred_cv_label.extend(pred_cv_label.astype(int))
            print('###',features_key,scaler,'[GMM]','Classification report for Cross Validation dataset')
            print_classification_report(cv_label, pred_cv_label)

            # [GMM] Test
            p_test = gmm.score_samples(test_df)
            pred_label = (p_test < epsilon)
            gmm_test_label.extend(test_label.astype(int))
            gmm_pred_test_label.extend(pred_test_label.astype(int))
            print('###',features_key,scaler,'[GMM]','Classification report for Test dataset')
            print_classification_report(test_label, pred_test_label)    
        except:                        
            print("### [GMM] Error!!")

#     # [MGM] save results
#     np.savetxt('output/mgm_cv_label_robust.out', mgm_cv_label, delimiter=',')
#     np.savetxt('output/mgm_pred_cv_label_robust.out', mgm_pred_cv_label, delimiter=',')
#     np.savetxt('output/mgm_test_label_robust.out', mgm_test_label, delimiter=',')
#     np.savetxt('output/mgm_pred_test_label_robust.out', mgm_pred_test_label, delimiter=',')
    
#     # [GMM] save results
#     np.savetxt('output/gmm_cv_label_robust.out', gmm_cv_label, delimiter=',')
#     np.savetxt('output/gmm_pred_cv_label_robust.out', gmm_pred_cv_label, delimiter=',')
#     np.savetxt('output/gmm_test_label_robust.out', gmm_test_label, delimiter=',')
#     np.savetxt('output/gmm_pred_test_label_robust.out', gmm_pred_test_label, delimiter=',')
    
    # [MGM] print results
    print('###',features_key,scaler,'[MGM]','Total Classification report for Cross Validation dataset')
    print_classification_report(mgm_cv_label, mgm_pred_cv_label)
    print('###',features_key,scaler,'[MGM]','Total Classification report for Test dataset')
    print_classification_report(mgm_test_label, mgm_pred_test_label)
    
    # [GMM] print results
    print('###',features_key,scaler,'[GMM]','Total Classification report for Cross Validation dataset')
    print_classification_report(gmm_cv_label, gmm_pred_cv_label)
    print('###',features_key,scaler,'[GMM]','Total Classification report for Test dataset')
    print_classification_report(gmm_test_label, gmm_pred_test_label)

### Directory:  b'/media/thiago/ubuntu/datasets/network/stratosphere-botnet-2011/ctu-13/raw/'
### Files:  [b'capture20110810.binetflow', b'capture20110811.binetflow', b'capture20110812.binetflow', b'capture20110815-2.binetflow', b'capture20110815-3.binetflow', b'capture20110815.binetflow', b'capture20110816-2.binetflow', b'capture20110816-3.binetflow', b'capture20110816.binetflow', b'capture20110817.binetflow', b'capture20110818-2.binetflow', b'capture20110818.binetflow', b'capture20110819.binetflow']
### Sample File:  /media/thiago/ubuntu/datasets/network/stratosphere-botnet-2011/ctu-13/pkl/capture20110810.binetflow


/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.

/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.



### drop_features03 RobustScaler [MGM] Classification report for Cross Validation dataset
### Binary F1 Score, Recall and Precision:
	F1 Score 0.111344
	Precision Score 0.067488
	Recall Score 0.317969
### drop_features03 RobustScaler [MGM] Classification report for Test dataset
### Binary F1 Score, Recall and Precision:
	F1 Score 0.107488
	Precision Score 0.064581
	Recall Score 0.320264
### drop_features03 RobustScaler [GMM] Classification report for Cross Validation dataset
### Binary F1 Score, Recall and Precision:
	F1 Score 0.068530
	Precision Score 0.035481
	Recall Score 1.000000
### drop_features03 RobustScaler [GMM] Classification report for Test dataset
### Binary F1 Score, Recall and Precision:
	F1 Score 0.107488
	Precision Score 0.064581
	Recall Score 0.320264
### Break File Loop!!
### drop_features03 RobustScaler [MGM] Total Classification report for Cross Validation dataset
### Binary F1 Score, Recall and Precision:
	F1 Score 0.111344
	Precision Score 0.067488
	Recall Score 

In [ ]:
# print("\nData Shape: ", df.shape)
# print("\nData Types: ", df.dtypes)
# df.head()
# data overview
# print("\nProto:\n", df['Proto'].value_counts())
# print("\nSrcAddr:\n", df['SrcAddr'].value_counts())
# print("\nDstAddr:\n", df['DstAddr'].value_counts())
# print("\nDport:\n", df['Dport'].value_counts())
# print("\nState:\n", df['State'].value_counts())
# print("\nsTos:\n", df['sTos'].value_counts())
# print("\ndTos:\n", df['dTos'].value_counts())
# print("\nLabel:\n", df['Label'].value_counts())

In [ ]:
# print("\nData Shape: ", df.shape)
# print("\nData Types: ", df.dtypes)
# df.head()

In [ ]:
# %%time
# v_features = [
#     'Dur',
#     'Proto',
#     'Dir',
#     'Dport',
#     'State',
#     'TotPkts'
# #     ,
# #     'TotBytes',
# #     'SrcBytes'
# #     'SrcAddr0',
# #     'DstAddr0',
# #     'DstAddr1',
# #     'DstAddr2',
# #     'DstAddr3'
# ]

# nplots=np.size(v_features)
# plt.figure(figsize=(15,4*nplots))
# gs = gridspec.GridSpec(nplots,1)
# for i, cn in enumerate(df[v_features]):
#     ax = plt.subplot(gs[i])
#     # print(cn)
#     sns.distplot(df[cn][df.Label == 1], bins=10, label='anomaly', color='r')
#     sns.distplot(df[cn][df.Label == 0], bins=10, label='normal', color='b')
#     ax.set_xlabel('')
#     ax.set_title('feature: ' + str(cn))
#     plt.legend()
# plt.savefig('distplot_df.png')
# plt.show()

In [ ]:
# %%time
# corr = df.corr()
# fig, ax = plt.subplots(figsize=(15,10))
# sns_plot = sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, ax=ax)
# figure = sns_plot.get_figure()
# figure.savefig('figures/corr_df.png', dpi=400)

In [ ]:
# %%time
# # drops N/A values
# tmp_df = df.dropna()

# # Create a scatter matrix of the aggregated dataframe
# # choose a few interesting features to pairplot based on the heat maps
# plot_features = ['Dur','Proto','Dir','Dport','State','sTos','dTos','TotPkts','Label']
# sns_plot = sns.pairplot(tmp_df, vars=plot_features, hue='Label')
# sns_plot.savefig("pairplot_df.png")

In [ ]:
# # The datastructure to hold our feature extraction functions, 
# # which will get applied to each aggregation of the datasets.
# extractors = {
#     'Label'   : [label_atk_v_norm, background_flow_count, normal_flow_count, n_conn,],
#     'Dport'   : [n_dports_gt1024, n_dports_lt1024],
#     'Sport'   : [n_sports_gt1024, n_sports_lt1024,],
#     'Dur'     : [avg_duration,],
#     'SrcAddr' : [n_s_a_p_address, n_s_b_p_address, n_s_c_p_address, n_s_na_p_address,],
#     'DstAddr' : [n_d_a_p_address, n_d_b_p_address, n_d_c_p_address, n_d_na_p_address,],
#     'Proto'   : [n_tcp, n_icmp, n_udp,],
# }
    
# # resample grouped by 1 second bin. must have a datetime-like index.
# r = df.resample('1S')
# n_df = r.agg(extractors) ## aggretation by data and functions specified by extractors

# n_df.columns = n_df.columns.droplevel(0) # get rid of the heirarchical columns
# pd.options.display.max_columns = 99

In [ ]:
# print('New nData Types: ', n_df.dtypes)
# n_df.head()

In [ ]:
# %%time
# n_features = ['background_flow_count','normal_flow_count','n_conn','n_dports>1024','n_dports<1024','n_s_a_p_address','n_s_b_p_address','n_s_c_p_address','n_s_na_p_address','n_d_a_p_address','n_d_b_p_address','n_d_c_p_address']
# plt.figure(figsize=(15,4*nplots))
# gs = gridspec.GridSpec(nplots,1)
# for i, cn in enumerate(n_df[n_features]):
#     ax = plt.subplot(gs[i])
# #     print(cn)
#     sns.distplot(n_df[cn][n_df.label == 1], bins=10, label='anomaly', color='r')
#     sns.distplot(n_df[cn][n_df.label == 0], bins=10, label='normal', color='b')
#     ax.set_xlabel('')
#     ax.set_title('feature: ' + str(cn))
#     plt.legend()
# plt.show()

In [ ]:
# %%time
# corr = n_df.corr()
# fig, ax = plt.subplots(figsize=(15,10))
# sns_plot = sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, ax=ax)
# figure = sns_plot.get_figure()
# figure.savefig('corr_df.png', dpi=400)

In [ ]:
# %%time
# # drops N/A values
# tmp_df = n_df.dropna()

# # Create a scatter matrix of the aggregated dataframe
# # choose a few interesting features to pairplot based on the heat maps
# plot_features = ['avg_duration','n_udp','background_flow_count','n_conn','n_icmp']
# sns.pairplot(tmp_df, vars=plot_features, hue='label')

In [ ]:
# %%time

# # visualize p_cv 
# p_cv_df = pd.DataFrame(data={'Prob':p_cv,'Label':cv_label})
# p_cv_df = p_cv_df.set_index(cv_df.index)
# p_cv_df.head()

# # plot distribution
# plt.figure(figsize=(17,5))
# sns.distplot(p_cv_df['Prob'][p_cv_df['Label'] == 1], bins=10, label='anomaly', color='r')
# sns.distplot(p_cv_df['Prob'][p_cv_df['Label'] == 0], bins=10, label='normal', color='b')
# plt.ylabel('Amount')
# plt.legend(loc='upper right')
# plt.show()

# # plot timeseries
# plt.figure(figsize=(17,5))
# anom_p_cv_df = p_cv_df[p_cv_df["Label"] == 1]
# norm_p_cv_df = p_cv_df[p_cv_df["Label"] == 0]
# anom_p_cv_df['Prob'].plot(style='g.', label='anomaly')
# norm_p_cv_df['Prob'].plot(style='y.', label='normal')
# plt.ylabel('Probability')
# plt.legend(loc='upper right')
# plt.show()

# # count probabilities
# print("Anomalies:\n", anom_p_cv_df['Prob'].value_counts())
# print("Normal:\n", norm_p_cv_df['Prob'].value_counts())

In [ ]:
# # visualize p_test 
# p_test_df = pd.DataFrame(data={'Prob':p_test,'Label':test_label})
# p_test_df = p_test_df.set_index(test_df.index)
# p_test_df.head()

# # plot distribution
# plt.figure(figsize=(17,5))
# sns.distplot(p_test_df['Prob'][p_test_df['Label'] == 1], bins=10, label='anomaly', color='r')
# sns.distplot(p_test_df['Prob'][p_test_df['Label'] == 0], bins=10, label='normal', color='b')
# plt.ylabel('Amount')
# plt.legend(loc='upper right')
# plt.show()

# # plot timeseries
# plt.figure(figsize=(17,5))
# anom_p_test_df = p_test_df[p_test_df["Label"] == 1]
# norm_p_test_df = p_test_df[p_test_df["Label"] == 0]
# norm_p_test_df['Prob'].plot(style='y.', label='normal')
# anom_p_test_df['Prob'].plot(style='g.', label='anomaly')
# plt.ylabel('Probability')
# plt.legend(loc='upper right')
# plt.show()

# # count probabilities
# print("Anomalies:\n", anom_p_test_df['Prob'].value_counts())
# print("Normal:\n", norm_p_test_df['Prob'].value_counts())

In [ ]:
# import sys
# print(sys.version)

# import matlab.engine
# eng = matlab.engine.start_matlab()
# print(eng.sqrt(4.))